HEART DISEASE


In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


df= pd.read_csv(r'C:\Users\Pranshu Saini\Desktop\disease-prediction-main\docpat\datasets\heart.csv')
df.head(15)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [2]:
df.shape

(303, 14)

In [3]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [4]:
# removing highly correlated features

corr_matrix = df.corr().abs() 

mask = np.triu(np.ones_like(corr_matrix, dtype = bool))
tri_df = corr_matrix.mask(mask)

to_drop = [x for x in tri_df.columns if any(tri_df[x] > 0.92)]

df = df.drop(to_drop, axis = 1)

print(f"The reduced dataframe has {df.shape[1]} columns.")

The reduced dataframe has 14 columns.


In [5]:
df.shape

(303, 14)

In [6]:
A = df.drop(columns='target', axis=1)
B = df['target']
A_training, A_testing, B_training, B_testing = train_test_split(A, B, test_size=0.2, stratify=B, random_state=2)

print(A.shape, A_training.shape, A_testing.shape)

(303, 13) (242, 13) (61, 13)


LogisticRegression

In [7]:
# fitting data to model

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(A_training, B_training)
B_pred = log_reg.predict(A_testing)
# accuracy score

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(B_training, log_reg.predict(A_training)))

log_reg_acc = accuracy_score(B_testing, log_reg.predict(A_testing))
print(log_reg_acc)

0.8512396694214877
0.819672131147541


c:\Users\Pranshu Saini\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K Neighbors Classifier (KNN)


In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(A_training, B_training)
# model predictions 

B_pred = knn.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, knn.predict(A_training)))

knn_acc = accuracy_score(B_testing, knn.predict(A_testing))
print(knn_acc)

0.78099173553719
0.6229508196721312


Support Vector Machine (SVM)

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC(probability=True)
parameters = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 15, 20]
}

grid_search = GridSearchCV(svc, parameters)
grid_search.fit(A_training, B_training)
# best parameters

grid_search.best_params_



{'C': 20, 'gamma': 0.0001}

In [10]:
# best score 

grid_search.best_score_



0.6981292517006803

In [11]:
svc = SVC(C = 10, gamma = 0.01, probability=True)
svc.fit(A_training, B_training)
# model predictions 

B_pred = svc.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, svc.predict(A_training)))

svc_acc = accuracy_score(B_testing, svc.predict(A_testing))
print(svc_acc)
# classification report

print(classification_report(B_testing, B_pred))

1.0
0.5409836065573771
              precision    recall  f1-score   support

           0       0.50      0.43      0.46        28
           1       0.57      0.64      0.60        33

    accuracy                           0.54        61
   macro avg       0.53      0.53      0.53        61
weighted avg       0.54      0.54      0.54        61



DECISION TREE

In [12]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(A_training, B_training)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [13]:

dtc = DecisionTreeClassifier(criterion= 'entropy', max_depth= 19, min_samples_leaf= 4, min_samples_split= 6, splitter= 'random')
dtc.fit(A_training, B_training)

DecisionTreeClassifier(criterion='entropy', max_depth=19, min_samples_leaf=4,
                       min_samples_split=6, splitter='random')

In [14]:
B_pred = dtc.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, dtc.predict(A_training)))

dtc_acc = accuracy_score(B_testing, dtc.predict(A_testing))
print(dtc_acc)

0.9132231404958677
0.6885245901639344


In [15]:
# classification report

print(classification_report(B_testing, B_pred))

              precision    recall  f1-score   support

           0       0.67      0.64      0.65        28
           1       0.71      0.73      0.72        33

    accuracy                           0.69        61
   macro avg       0.69      0.69      0.69        61
weighted avg       0.69      0.69      0.69        61



In [16]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'SVM', 'Decision Tree Classifier'],
    'Score': [100*round(log_reg_acc,4), 100*round(knn_acc,4), 100*round(svc_acc,4), 100*round(dtc_acc,4)]
})
models.sort_values(by = 'Score', ascending = False)

,Model,Score
0,Logistic Regression,81.97
3,Decision Tree Classifier,68.85
1,KNN,62.30
2,SVM,54.10


In [17]:
filtered_df = df[df['target'] == 0]
C = pd.DataFrame(filtered_df)   
print(C)


     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
165   67    1   0       160   286    0        0      108      1      1.5   
166   67    1   0       120   229    0        0      129      1      2.6   
167   62    0   0       140   268    0        0      160      0      3.6   
168   63    1   0       130   254    0        0      147      0      1.4   
169   53    1   0       140   203    1        0      155      1      3.1   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
165      1   3     2       0  
166      1   2     3     

In [ ]:
import numpy as np

a = [63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 0, 0, 1]
b=[67  ,  1,   0,       160,   286,    0,        0,      108,      1,      1.5,  1,   3,     2 ]
a_reshaped = np.array().reshape(1, -1)

B = log_reg.predict(a_reshaped)
print(B)


[1]


c:\Users\Pranshu Saini\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [20]:
import pickle
filename = r'C:\Users\Pranshu Saini\Desktop\disease-prediction-main\docpat\model\heart_disease_model.pkl'
pickle.dump(log_reg, open(filename, 'wb'))

In [ ]:
'''import pickle
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
heart_model = load_model(r'C:\Users\DELL\Desktop\app\heart_disease_model.pkl')
def predict(inputs):
    return heart_model.predict(inputs)'''